In [551]:
!pip install ace_tools spacy tabula https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.5.0/en_core_web_sm-3.5.0-py3-none-any.whl


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 12.9 MB/s eta 0:00:0000:0100:01


In [552]:
# Packages
import os
import re
import pdfplumber
import numpy
import pandas as pd
import spacy
from datetime import datetime
from dateutil.parser import parse

In [553]:
# Load spaCy English model
nlp = spacy.load("en_core_web_sm")

# Load the provider info CSV for ending keywords
provider_info = pd.read_csv('provider.csv')

# Load the company info CSV for ticker validation and company metadata
company_info = pd.read_csv('company_info.csv')  # Replace with the actual path

company_info = company_info.drop_duplicates(subset='Ticker Symbol')

# Create a dictionary to map ticker symbols to company name and industry
ticker_map = company_info.set_index('Ticker Symbol')[['Company Name', 'Industry']].to_dict(orient='index')

In [554]:
provider_info

,Provider,file_name,Ending,Price Target,Recommendation
0,BGC Partners,BGC,Disclosures Appendix,Price Target ($) {price},"{RATING} ({TICKER},$"
1,Needham,Needham,Analyst Certification,Price Target ${price},Stock Rating {RATING}
2,BTIG,BTIG,Appendix: Analyst Certification and Other Impo...,12 month target ${price},{RATING} 52 week range
3,Wells Fargo,Wells_Fargo,Required Disclosures,Overweight/${price},{Rating}/$
4,Barclays,Barclays,ANALYST(S) CERTIFICATION(S),Price Target USD {price},Stock Rating {RATING}
5,JP Morgan,JP_Morgan,Analyst Certification,Price Target ({Letters & numbers}): ${price},{Rating} {TICKER}
6,Brean Capital,Brean_Capital,Analyst Certification,PT: $ {price},{Rating} PT:$
7,Hilliard Lyons,Hilliard_Lyons,Analyst Certification,Price Target {price},NSYE -- {Rating} --
8,Alliance Global Partners,Alliance_Global_Partners,Imporant Research Disclosures,Price Target ${price},{Rating} (Ticker:
9,Mizuho Securities,Mizuho_Securities,IMPORTANT DISCLOSURES,Price Target ${price},Rating Buy


In [555]:
df_missing_paragraphs.value_counts("provider")

Series([], Name: count, dtype: int64)

In [556]:
df_missing_paragraphs

,document_id,filename,date,provider,ticker,company_name,industry,paragraphs,target_price,rating


In [557]:
# Only relevant to check font type and size for
import pdfplumber

def extract_words_with_formatting(page):

    # Extract words with their bounding boxes
    words = page.extract_words(extra_attrs=["fontname", "size"])

    formatted_words = []
    for word in words:
        formatted_words.append({
            "word": word["text"],
            "font": word.get("fontname", "Unknown"),
            "size": word.get("size", "Unknown"),
            "x0": word["x0"],
            "x1": word["x1"],
            "top": word["top"],
            "bottom": word["bottom"]
        })
    return formatted_words


# Example usage with pdfplumber
pdf_path = "/Users/oskarroeske/Desktop/Analyst_Reports/Production/20201113_Barclays_WMT_WMT-_F3Q21_Earnings_Preview.pdf"

with pdfplumber.open(pdf_path) as pdf:
    for page_number, page in enumerate(pdf.pages, start=1):
        print(f"Page {page_number}:")
        formatted_words = extract_words_with_formatting(page)
        for word_info in formatted_words:
            print(
                f"Word: '{word_info['word']}', Font: {word_info['font']}, Size: {word_info['size']}, "
                f"Position: ({word_info['x0']}, {word_info['top']} - {word_info['x1']}, {word_info['bottom']})"
            )


Page 1:
Word: 'CORE', Font: AAAAAA+DejaVuSans, Size: 9.234802439216537, Position: (550.1367164631358, 5.729781421702796 - 576.1050892464455, 14.964583860919333)
Word: 'Equity', Font: AAAAAA+DejaVuSans, Size: 8.524433020815309, Position: (431.92680543229903, 47.47404369740127 - 458.88199504660076, 55.99847671821658)
Word: 'Research', Font: AAAAAA+DejaVuSans, Size: 8.524433020815309, Position: (461.5916658625593, 47.47404369740127 - 500.69250799390153, 55.99847671821658)
Word: '|', Font: AAAAAA+DejaVuSans, Size: 8.524433020815309, Position: (503.40217880986006, 47.47404369740127 - 506.274180148101, 55.99847671821658)
Word: 'Instant', Font: AAAAAA+DejaVuSans, Size: 8.524433020815309, Position: (508.98385096405957, 47.47404369740127 - 538.652873619231, 55.99847671821658)
Word: 'Insights', Font: AAAAAA+DejaVuSans, Size: 8.524433020815309, Position: (541.3625444351894, 47.47404369740127 - 574.6860847836816, 55.99847671821658)
Word: '6', Font: AAAAAA+DejaVuSans, Size: 7.814063602413967, Posit

In [568]:
# Test new Structure

patterns = {
    "APP Securities Pty Ltd": {
        "price_patterns": {
            "primary": r"TARGET PRICE (NA|\$(\d+(\.\d+)?))",
        },
        "rating_patterns": {
            "primary": r"\b(BUY|HOLD|SELL|OVERWEIGHT|UNDERPERFORM)\b",
        },
        "ending_patterns": [r"Analyst Certification"],
        "font_patterns": [
            {"font_type": r"[A-Z]+[+]Calibri(,Bold)?", "font_size": 8.04},
            {"font_type": r"[A-Z]+[+]Calibri(,Bold)?", "font_size": 8.25},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "Alliance Global Partners": {
        "price_patterns": {
            "primary": r"Price Target (NA|\$(\d+(\.\d+)?))",
        },
        "rating_patterns": {
            "primary": r"\b(Buy|Hold|Sell|Overweight|Underperform)\b",
        },
        "ending_patterns": [r"Imporant Research Disclosures"],
        "font_patterns": [
            {"font_type": r"[A-Z]+[+]ArialMT(-BoldMT)?", "font_size": 8.00},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "Barclays": {
        "price_patterns": {
            "primary": r"Price Target USD (\d+(\.\d+)?)",
            "secondary": r"Price Target: USD (\d+(\.\d+)?)",
        },
        "rating_patterns": {
            "primary": r"Stock Rating ([A-Za-z]+)",
            "secondary": r"\b(BUY|HOLD|SELL|OVERWEIGHT|OVERPERFORM|UNDERPERFORM|UNDERWEIGHT|NEUTRAL)\b",
            "tertiary": r"\b(Buy|Hold|Sell|Overweight|Overperform|Underperform|Underweight|Neutral)\b",
        },
        "ending_patterns": [r"ANALYST\(S\) CERTIFICATION\(S\)",
                            r"Analyst\(s\) Certification\(s\)"],
        "font_patterns": [
            {"font_type": r"[A-Z]+[+]Expert Sans (Extra Bold)?(Regular)?(Regular,Bold)?", "font_size": 9.0},
            {"font_type": r"[A-Z]+[+]Expert Sans (Extra Bold)?(Regular)?", "font_size": 8.04},
            {"font_type": r"[A-Z]+[+]SourceSansPro(-Bold)?(-Regular)?", "font_size": 9.00},
            {"font_type": r"[A-Z]+[+]DejaVuSans(-Bold)?", "font_size": 9.01},
            {"font_type": r"[A-Z]+[+]DejaVuSans(-Bold)?", "font_size": 7.58},
            {"font_type": r"[A-Z]+[+]DejaVuSans(-Bold)?", "font_size": 7.52},
            {"font_type": r"[A-Z]+[+]DejaVuSans(-Bold)?", "font_size": 8.52},
            {"font_type": r"[A-Z]+[+]DejaVuSans(-Bold)?", "font_size": 8.75},
            {"font_type": r"[A-Z]+[+]DejaVuSans(-Bold)?", "font_size": 7.60},
            {"font_type": r"[A-Z]+[+]DejaVuSans(-Bold)?", "font_size": 7.11},
            {"font_type": r"[A-Z]+[+]DejaVuSans(-Bold)?", "font_size": 6.52},
            {"font_type": r"[A-Z]+[+]DejaVuSans(-Bold)?", "font_size": 7.02},
            {"font_type": r"[A-Z]+[+]DejaVuSans(-Bold)?", "font_size": 7.03},
        ]
    },
    "BGC Partners": {
        "price_patterns": {
            "primary": r"Price Target \(\$\) (\d+(\.\d+)?)",
        },
        "rating_patterns": {
            "primary": r"([A-Za-z]+) \(\w+,",
        },
        "ending_patterns": [r"Disclosures Appendix"],
        "font_patterns": [
            {"font_type": r"Tahoma(-Bold)?", "font_size": 7.92},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "Brean Capital": {
        "price_patterns": {
            "primary": r"PT: \$ (\d+(\.\d+)?)",
            "secondary": r"TP: \$(\d+(\.\d+)?)",

        },
        "rating_patterns": {
            "primary": r"\b(Buy|Hold|Sell|Overweight|Underperform)\b"
        },
        "ending_patterns": [r"Analyst Certification",r"Important Disclosures"],
        "font_patterns": [
            {"font_type": r"Tahoma(-Bold)?", "font_size": 7.92},
            {"font_type": r"[A-Z]+[+]Calibri(-Bold)?(-Italic)?", "font_size": 8.00},
            {"font_type": r"[A-Z]+[+]Calibri(-Bold)?(-Italic)?", "font_size": 9.00},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "BTIG": {
        "price_patterns": {
            "primary": r"\$(\d+(\.\d+)?) 12 month target ",
        },
        "rating_patterns": {
            "primary": r"\b(BUY|HOLD|SELL|OVERWEIGHT|UNDERPERFORM)\b",
        },
        "ending_patterns": [r"Appendix: Analyst Certification and Other Important Disclosures"],
        "font_patterns": [
            {"font_type": r"[A-Z]+[+]Corbel(,Bold)?(,-Italic)?", "font_size": 9.96},
            {"font_type": r"[A-Z]+[+]Calibri(,Bold)?(,Italic)?", "font_size": 9.96},
            {"font_type": r"[A-Z]+[+]Calibri(-Bold)?(-Italic)?", "font_size": 10.0},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "Cascend Securities -Historical-": {
        "price_patterns": {
            "primary": r"Price target: \$(\d+(\.\d+)?)",
        },
        "rating_patterns": {
            "primary": r"Rating: ([A-Za-z]+)",
        },
        "ending_patterns": [r"Disclosures: "],
        "font_patterns": [
            {"font_type": r"[A-Z]+[+]Calibri(,Bold)?", "font_size": 12.0},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "Deutsche Bank": {
        "price_patterns": {
            "primary": r"Price Target \(USD\) (\d+(\.\d+)?)",
            "secondary": r"Price target (\d+(\.\d+)?)",
        },
        "rating_patterns": {
            "primary": r"\b(Buy|Hold|Sell|Overweight|Underperform|Underweight|Neutral)\b",
            #"secondary": r"Rating ([A-Za-z]+)",
        },
        "ending_patterns": [r"Appendix 1"],
        "font_patterns": [
            {"font_type": r"[A-Z]+[+]UniversDeutscheBank-Regular", "font_size": 9.0},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "FinTrust Investment Advisors": {
        "price_patterns": {
            "primary": r"Target Price: \$(\d+(,\d+)?)",
        },
        "rating_patterns": {
            "primary": r"Fintrust Rating: ([A-Za-z]+)",
        },
        "ending_patterns": [r"Important Disclosures:"],
        "font_patterns": [
            {"font_type": r"Arial(-BoldMT)?", "font_size": 7.92},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "Gilford Securities Inc": {
        "price_patterns": {
            "primary": r"\, \$(\d+(\.\d+)?)",
        },
        "rating_patterns": {
            "primary": r"Rated: ([A-Za-z]+)",
            "secondary": r"\b(BUY|HOLD|SELL|OVERWEIGHT|UNDERPERFORM)\b",
            "tertiary": r"\b(Buy|Hold|Sell|Overweight|Underperform|Underweight|Neutral)\b",
        },
        "ending_patterns": [r"ANALYST CERTIFICATION",r"REQUIRED DISCLOSURES"],
        "font_patterns": [
            {"font_type":  r"Arial(MT)?(-BoldMT)?", "font_size": 10.02},
            {"font_type":  r"Arial(MT)?(-BoldMT)?", "font_size": 10.01},
            {"font_type": "Not Available", "font_size": 10.00},
        ]
    },
    "Hilliard Lyons": {
        "price_patterns": {
            "primary": r"Price Target (NA|\$(\d+(\.\d+)?))",
        },
        "rating_patterns": {
            "primary": r"-- NYSE\s+[–\-—]+\s+([A-Za-z]+)\s+[–\-—]+",
            "secondary":r"NYSE\s+[–\-—]+\s+([A-Za-z\- ]+?)(?=\s*[-–—]\d)"
        },
        "ending_patterns": [r"Analyst Certification"],
        "font_patterns": [
            {"font_type": r"Verdana(-Bold)?", "font_size": 9.00},
            {"font_type":r"TimesNewRomanPS(-BoldMT)?", "font_size": 10.98},
            {"font_type":r"Times(-Bold)?", "font_size": 10.98},
            ]
    },
    "IBI Investment House": {
        "price_patterns": {
            "primary": r"Price target: \$(\d+(,\d+)?)",
        },
        "rating_patterns": {
            "primary": r"Recommendation: ([A-Za-z]+)",
        },
        "ending_patterns": [r"Disclosures"],
        "font_patterns": [
            {"font_type": r"Tahoma(-Bold)?", "font_size": 7.92},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "IFS Securities": {
        "price_patterns": {
            "primary": r"Price Target: \$(\d+(,\d+)?)",
        },
        "rating_patterns": {
            "primary": r"(Buy|Hold|Sell|Overweight|Underperform|Outperform)",
        },
        "ending_patterns": [r"Important Investor Disclosures"],
        "font_patterns": [
            {"font_type": r"Arial(-BoldMT)?", "font_size": 11.04},
            {"font_type":r"TimesNewRomanPS(MT)?(-BoldMT)?", "font_size": 11.04},
        ]
    },
    "JP Morgan": {
        "price_patterns": {
            "primary": r"Price Target: \$(\d+(\.\d+)?)",
            "secondary": r"Price Target \([A-Za-z0-9\-]+\): \$(\d+(\.\d+)?)"
        },
        "rating_patterns": {
            "primary": r"\b(Buy|Hold|Sell|Overweight|Underperform|Underweight|Neutral)\b",
        },
        "ending_patterns": [r"Analyst Certification"],
        "font_patterns": [
            {"font_type": r"[A-Z]+[+]TimesNewRoman(,Bold)?(,Italic)?", "font_size": 9.60},
            {"font_type": r"[A-Z]+[+]TimesNewRoman(,Bold)?(,Italic)?", "font_size": 10.08},
            {"font_type": r"[A-Z]+[+]TimesNewRomanPS(MT)?(-BoldMT)(-ItalicMT)?", "font_size": 11.04},
            {"font_type": r"[A-Z]+[+]TimesNewRomanPS(MT)?(-BoldMT)?(-ItalicMT)?", "font_size": 10.00},
            {"font_type": r"[A-Z]+[+]TimesNewRoman(,Bold)?(,Italic)?", "font_size": 10.56},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "Mizuho Securities": {
        "price_patterns": {
            "primary": r"Price Target \$(\d+(\.\d+)?)",
        },
        "rating_patterns": {
            "primary": r"Rating ([A-Za-z]+)",
        },
        "ending_patterns": [r"IMPORTANT DISCLOSURES"],
        "font_patterns": [
            {"font_type": r"Tahoma(-Bold)?", "font_size": 7.92},
            {"font_type": r"Times-Roman(-Bold)?(-Italic)?(-BoldItalic)?", "font_size": 10.50},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "Needham": {
        "price_patterns": {
            "primary": r"Price Target: \$(\d{1,3}(,\d{3})*(\.\d{2})?)",
            "secondary": r"PRICE TARGET: \$(\d{1,3}(,\d{3})*(\.\d{2})?)",
        },
        "rating_patterns": {
            "primary": r"\b(BUY|HOLD|SELL|OVERWEIGHT|UNDERPERFORM)\b",
            "secondary": r"Rating (\w+)",
        },
        "ending_patterns": [r"Analyst Certification"
                            r"ANALYST CERTIFICATION"],
        "font_patterns": [
            {"font_type": r"[A-Z]+[+]Cambria(-Bold)?(-Regular)?", "font_size": 10.0},
            {"font_type": r"[A-Z]+[+]KeplerStd(-Bold)?", "font_size": 9.0},
        ]
    },
    "Phillip Securities": {
        "price_patterns": {
            "primary": r"TARGET PRICE USD (\d+(\.\d+)?)",
            "secondary": r"U\s?S\s?D\s?(\d+(?:\s?\.\s?\d+)?)\s?T\s?A\s?R\s?G\s?E\s?T\s?P\s?R\s?I\s?C\s?E",
        },
        "rating_patterns": {
            "primary": r"\b(BUY|HOLD|SELL|OVERWEIGHT|UNDERPERFORM)\b",
            "secondary": r"\b(B U Y|H O L D|S E L L|O V E R W E I G H T|U N D E R P E R F O R M| N E U T R A L | O U T P E R F O R M)\b",
        },
        "ending_patterns": [r"Contact Information"],
        "font_patterns": [
            {"font_type": r"[A-Z]+[+]Calibri(-Bold)?", "font_size": 9.96},
            {"font_type": r"[A-Z]+[+]Calibri(-Bold)?", "font_size": 10.0},
        ]
    },
    "Pivotal Research Group": {
        "price_patterns": {
            "primary": r"Target Price: \$(\d+(,\d+)?)",
        },
        "rating_patterns": {
            "primary": r"RATING: ([A-Za-z]+)",
        },
        "ending_patterns": [r"Appendix: Important Disclosures"],
        "font_patterns": [
            {"font_type": r"[A-Z]+[+]Helvetica(-Bold)?", "font_size": 9.96},
            {"font_type": r"Arial(MT)?(-BoldMT)?", "font_size": 8.04},
            {"font_type": r"Arial(MT)?(-BoldMT)?", "font_size": 8.03},
            {"font_type": r"Arial(MT)?(-BoldMT)?", "font_size": 9.00},
        ]
    },
    "Spartan Capital Securities LLC": {
        "price_patterns": {
            "primary": r"T \$(\d+(\.\d+)?)",
        },
        "rating_patterns": {
            "primary": r"([A-Za-z]+)",
        },
        "ending_patterns": [r"Important Disclosures"],
        "font_patterns": [
            {"font_type": r"Tahoma(-Bold)?", "font_size": 7.92},
            {"font_type": "Not Available", "font_size": 10.0},
        ]
    },
    "Wells Fargo": {
        "price_patterns": {
            "primary": r"\/Price Target: \$(\d+(\.\d+)?)",
            "secondary": r"Price Target\/Prior: \$(\d+(\.\d+)?)",
            #"tertiary": r"\/\$(\d+(\.\d+)?)"
        },
        "rating_patterns": {
            "primary": r"\/\b(Buy|Hold|Sell|Overweight|Underperform|Underweight|Neutral)\b",
            "secondary": r"Rating (\w+)",
            "tertiary": r"([A-Za-z]+)/\$",
        },
        "ending_patterns": [r"Required Disclosures"],
        "font_patterns": [
            {"font_type": r"[A-Z]+[+]WellsFargoSans(-Light)?(,-SemiBold)?", "font_size": 9.00},
            {"font_type": r"[A-Z]+[+]Verdana(-Bold)?", "font_size": 8.04},
            {"font_type": r"[A-Z]+[+]Verdana(-Bold)?", "font_size": 9.88},
            {"font_type": r"[A-Z]+[+]DejaVuSans(-Bold)?", "font_size": 9.01}
        ]
    }
}



In [685]:
def check_validity(paragraph):
    # Parse the paragraph
    paragraph = re.sub(r"\s+", " ", paragraph).strip()
    doc = nlp(paragraph)
        
    for sent in doc.sents:
        has_verb = False
        has_subject = False
        
        for token in sent:
            # Check for a verb
            if token.pos_ in {"VERB", "AUX"}:
                has_verb = True
            # Check for a subject
            if token.dep_ in {"nsubj", "nsubjpass"}:
                has_subject = True
        
        # If both a verb and a subject are found, the sentence is valid
        if has_verb and has_subject:
            return True
        
        # At least one word with 5+ letters and all upper case
        if re.search(r"[A-Z]{5,}", paragraph):
            return True
    return False

def filter_valid_paragraphs(paragraphs):
    valid_paragraphs = []

    for paragraph in paragraphs:
        if check_validity(paragraph):  # Validate each paragraph
            valid_paragraphs.append(paragraph)
    return valid_paragraphs

# Function to check if a word ends a sentence
def is_sentence_end(word):
    text = word["text"]
    next_text = word.get("next_text", "")

    # Check for sentence-ending punctuation
    if text.endswith(".") or text.endswith("!") or text.endswith("?"):
        # Ensure it's not part of a decimal number
        if next_text:
            if text.replace(".", "").isdigit() and next_text.isdigit():
                return False
            # Check if the next word starts with an uppercase letter (for ".")
            if text.endswith(".") and next_text and not next_text.istitle():
                return False
            return True
    return False

def extract_text_with_format(page, provider, page_number,url_date):

    # Ensure url_date is a datetime.date object
    if isinstance(url_date, datetime):
        url_date = url_date.date()

    date = None

    # Access patterns
    provider_patterns = patterns[provider]
    price_patterns = provider_patterns["price_patterns"]
    rating_patterns = provider_patterns["rating_patterns"]
    ending_patterns = provider_patterns["ending_patterns"]
    font_patterns = provider_patterns["font_patterns"]

    # Extract words with font and size details
    words = page.extract_words(extra_attrs=["fontname", "size"])

    # Handle empty words
    if not words:
        return [], False, None, None, date

    # Round text sizes to 2 decimal places
    for word in words:
        if "size" in word and word["size"] is not None:
            word["size"] = round(word["size"], 2)

    # Sort words by vertical and horizontal position
    words.sort(key=lambda w: (w["top"], w["x0"]))

    paragraphs = []
    current_paragraph = []
    current_top = None
    sentence_ended = False  # Track if the last processed sentence has ended
    lookahead_buffer = []
    rating = None
    price = None
    extracted_date = None

    # Add next word context for sentence-ending logic
    for i in range(len(words) - 1):
        words[i]["next_text"] = words[i + 1]["text"]
    words[-1]["next_text"] = None  # Last word has no next word


    # Loop through the words to group into paragraphs
    for word in words:

        # Build lookahead buffer
        lookahead_buffer.append(word["text"])
        if len(lookahead_buffer) > 35:
            lookahead_buffer.pop(0)
        buffer_text = " ".join(lookahead_buffer)

        # Extract rating or price (only for page 1)
        if page_number == 1:
            # Extract rating
            if not rating:
                for pattern_key in ["primary", "secondary", "tertiary"]:
                    pattern = rating_patterns.get(pattern_key)
                    if pattern:
                        rating_match = re.search(pattern, buffer_text)
                        if rating_match:
                            rating = rating_match.group(1)
                            break

            # Extract price
            if not price:
                for pattern_key in ["primary", "secondary", "tertiary"]:
                    pattern = price_patterns.get(pattern_key)
                    if pattern:
                        price_match = re.search(pattern, buffer_text)
                        if price_match:
                            price = price_match.group(1)
                            break

            # Look for date entities in the text
            if len(buffer_text.split()) > 30:
                if date is None:
                    sequence = nlp(buffer_text)
                    for ent in sequence.ents:
                        if ent.label_ == "DATE":
                            try:
                                # Attempt to parse the date
                                candidate_date = parse(ent.text, fuzzy=True).date()
                        
                                # Validate the date (ensure it's within a reasonable range)
                                if datetime.min.date() <= candidate_date <= datetime.max.date():
                                    extracted_date = candidate_date
                                    break  # Break if a valid date is found

                            except (ValueError, OverflowError):
                                #print(f"Invalid or overflow date encountered: {ent.text}")  # Debugging
                                continue  # Skip invalid dates
                    
                    # Calculate the difference
                    date_difference = 0
                    if extracted_date:
                        date_difference = (url_date - extracted_date).days
                        
                    # Check the conditions
                    if extracted_date and (0 <= date_difference <= 10):  # extracted_date is within 10 days earlier
                        date = extracted_date
                        
        # Check for ending pattern
        for ending_pattern in ending_patterns:
            if re.search(ending_pattern, buffer_text):
                return filter_valid_paragraphs(paragraphs), True, rating, price, date
        
        # Match word against font patterns
        is_font_matched = False
        for font_pattern in font_patterns:
            font_type = font_pattern["font_type"]
            font_size = font_pattern["font_size"]
            if re.match(font_type, word["fontname"]) and word["size"] == font_size:
                is_font_matched = True
                break

        if not is_font_matched:
            continue

        # Check if the word ends a sentence
        if is_sentence_end(word):
            sentence_ended = True

        # Check if we need to start a new paragraph
        if sentence_ended and current_top is not None and abs(word["top"] - current_top) >= 13:
            # End the current paragraph and start a new one
            paragraphs.append(" ".join(current_paragraph))
            current_paragraph = [word["text"]]
            sentence_ended = False  # Reset sentence-ended flag
        else:
            # Continue the current paragraph
            current_paragraph.append(word["text"])

        # Update the current top position
        current_top = word["top"]

    # Add the last paragraph
    if current_paragraph:
        paragraphs.append(" ".join(current_paragraph))

    """ #Assign url_date in case no date was found on the first page
    if date is None:
        date = url_date"""

    return filter_valid_paragraphs(paragraphs), False, rating, price, date


# NEW VERSION (16.11.2024)
def extract_metadata(filename, ticker_map):
    
    # Extract the date (first 8 digits in the filename)
    date_match = re.match(r"(\d{8})", filename)
    if not date_match:
        return None, None, None, None, None
    date_str = date_match.group(1)
    date = datetime.strptime(date_str, "%Y%m%d")

    #print(f"fileName: {filename}")
    #print(f"date: {date}")

    # Look for the ticker in the filename
    for ticker in ticker_map.keys():
        ticker_pattern = f"_{ticker}_"  # Ensure ticker is surrounded by underscores
        if ticker_pattern in filename:
            # Extract the portion between date and ticker as the provider
            provider_section = filename.split(f"{date_str}_")[1].split(f"_{ticker}_")[0]
            provider = provider_section.replace('_', ' ')  # Replace underscores with spaces
            # Get company name and industry from the ticker_map
            company_name = ticker_map[ticker]['Company Name']
            industry = ticker_map[ticker]['Industry']
            return date, provider, ticker, company_name, industry

    # If no ticker is found, return None for ticker-related fields
    return date, None, None, None, None


In [ ]:

# Define function to process all PDFs in the directory and store data in DataFrame
def extract_text_from_all_pdfs_to_dataframe(directory_path, provider_info, ticker_map):
    data = []
    id_counter = 1  # Initialize an ID counter
    for filename in os.listdir(directory_path):
        if filename.endswith(".pdf"):  # Process only PDF files
            file_path = os.path.join(directory_path, filename)

            
            # Extract metadata from filename
            url_date, provider, ticker, company_name, industry = extract_metadata(filename,ticker_map=ticker_map)
            all_paragraphs = []  
            first_rating = None
            first_price = None
            first_date = None
            stop_extraction = False

            with pdfplumber.open(file_path) as pdf:
                for page_number, page in enumerate(pdf.pages, start=1):
                    if stop_extraction:
                        break  # Exit the loop if stop_extraction is set
            
                    # Extract data from the current page
                    paragraphs, stop_extraction, rating, price,new_date = extract_text_with_format(page, provider=provider,page_number=page_number,url_date=url_date)
                    
                    # Append paragraphs from the current page
                    all_paragraphs.extend(paragraphs)

                    # Capture the first non-None rating and price
                    if rating is not None and first_rating is None:
                        first_rating = rating
                    if price is not None and first_price is None:
                        first_price = price

                    if new_date is not None and first_date is None:
                        first_date = new_date
                        
            if first_date is None:
                first_date = url_date.date()
                        
            # Add extracted data to the list
            data.append({
                "document_id": id_counter,  # Unique ID
                "filename": filename,
                "date": first_date,
                "provider": provider,
                "ticker": ticker,
                "company_name": company_name,
                "industry": industry,
                "paragraphs": all_paragraphs,
                "target_price":first_price.replace(" ","") if first_price else None,
                "rating": first_rating.lower().replace(" ","") if first_rating else None
                })
            id_counter += 1  # Increment the ID counter for the next row

    # Create a DataFrame from the list of dictionaries
    df = pd.DataFrame(data)
    return df

# Directory path
pdf_directory = "/Users/oskarroeske/Desktop/Analyst_Reports/Production"

# Run the function and store results in a DataFrame
df_reports = extract_text_from_all_pdfs_to_dataframe(pdf_directory, provider_info, ticker_map)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname E identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


In [687]:
df_reports

,document_id,filename,date,provider,ticker,company_name,industry,paragraphs,target_price,rating
0,1,20140923_Brean_Capital_NKTR_NKTR-_Movantik_App...,2014-09-16,Brean Capital,NKTR,Nektar Therapeutics,Healthcare,"[jaschoff@breancapital.com Jonathan Aschoff , ...",20.00,buy
1,2,20140130_Needham_EBAY_EBAY-_4Q_Results-_Icahn_...,2014-01-23,Needham,EBAY,eBay Inc.,Consumer Discretionary,[HOLD NA INVESTMENT HIGHLIGHTS: eBay reported ...,None,hold
2,3,20140127_Needham_INTC_INTC-_Disappointing_Data...,2014-01-17,Needham,INTC,Intel Corp.,Technology,[HOLD INVESTMENT HIGHLIGHTS: NA Intel's 4Q13 r...,None,hold
3,4,20140310_Needham_TSLA_SCTY-_No_Full_Financials...,2014-03-03,Needham,TSLA,Tesla Inc.,Automobile,[HOLD NA INVESTMENT HIGHLIGHTS: SCTY once agai...,None,hold
4,5,20140529_Needham_CRM_CRM-_A_Great_Start_To_Lik...,2014-05-21,Needham,CRM,Salesforce.com Inc.,Technology,[BUY INVESTMENT HIGHLIGHTS: Q1 was rock solid ...,None,buy
5,6,20140324_Hilliard_Lyons_NKE_Report_received_in...,2014-03-24,Hilliard Lyons,NKE,Nike Inc.,Clothing,[Investment Highlights Fiscal 3Q results were ...,None,neutral
6,7,20140806_Pivotal_Research_Group_T_DTV-_Updatin...,2014-08-05,Pivotal Research Group,T,AT&T Inc.,Communication Services,[We updated our model post DTV's solid better ...,90,hold
7,8,20140307_Needham_CRM_CRM-_Announced_CFO_Depart...,2014-02-28,Needham,CRM,Salesforce.com Inc.,Technology,[BUY INVESTMENT HIGHLIGHTS: $80.00 Q4 was rock...,None,buy
8,9,20140725_Needham_GOOGL_GOOG-_Raising_2H14_Reve...,2014-07-18,Needham,GOOGL,Alphabet Inc.,Communication Services,[BUY $675.00 INVESTMENT HIGHLIGHTS: We believe...,None,buy
9,10,20140825_Gilford_Securities_Inc_FL_Report_rece...,2014-08-25,Gilford Securities Inc,FL,Foot Locker,Clothing,"[Investment Opinion: Profit growth, dividend g...",3.75,buy


In [647]:
df_missing_paragraphs = df_reports[df_reports["paragraphs"].apply(lambda x: x == [])]

In [648]:
df_missing_paragraphs

,document_id,filename,date,provider,ticker,company_name,industry,paragraphs,target_price,rating
3,4,20201001_Pivotal_Research_Group_AMZN_AMZN-_We_...,2020-09-30,Pivotal Research Group,AMZN,Amazon.com Inc.,Consumer Discretionary,[],"4,500",buy
115,116,20200821_Pivotal_Research_Group_FL_FL-_Recap_2...,2020-08-21,Pivotal Research Group,FL,Foot Locker,Clothing,[],39,buy
188,189,20200728_Pivotal_Research_Group_UAA_UAA-_Previ...,2020-07-28,Pivotal Research Group,UAA,Under Armour,Clothing,[],12,hold
795,796,20200226_Pivotal_Research_Group_FL_FL-_Preview...,2020-02-26,Pivotal Research Group,FL,Foot Locker,Clothing,[],45,buy
806,807,20190304_Pivotal_Research_Group_FL_FL-_Upgradi...,2019-03-04,Pivotal Research Group,FL,Foot Locker,Clothing,[],73,buy


In [ ]:
#df_reports.to_csv("reports.csv")

# Cleaning of paragraphs

In [649]:
import re

def clean_paragraph(paragraph):
    # Remove email addresses
    paragraph = re.sub(r"\S+@\S+", "", paragraph)
    
    # Remove phone numbers
    paragraph = re.sub(r"\b\d{1,3}[-.\s]?\d{3}[-.\s]?\d{4}\b", "", paragraph)
    
    # Remove URLs
    paragraph = re.sub(r"http\S+|www\S+", "", paragraph)
    
    # Remove special characters (keep alphanumerics, spaces, and common punctuation)
    paragraph = re.sub(r"[^\w\s,.!?%$:-]", "", paragraph)
    
    # Remove multiple spaces or newlines
    paragraph = re.sub(r"\s+", " ", paragraph).strip()
    
    # Remove short sentences (fewer than 6 words)
    if len(paragraph.split()) < 6:
        return None
    
    return paragraph

# Apply cleaning function
df_reports['paragraphs'] = df_reports['paragraphs'].apply(
    lambda paragraphs: [clean_paragraph(p) for p in paragraphs if clean_paragraph(p)]
)

# Drop rows where the `paragraphs` column is empty after cleaning
df_reports = df_reports[df_reports['paragraphs'].str.len() > 0]

df_reports["rating"] = df_reports["rating"].astype(str).apply(lambda x: x.lower() if x != "nan" else None)

/var/folders/2h/tyzkflxs465698k6z8l3ktj00000gn/T/ipykernel_7542/307161428.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reports["rating"] = df_reports["rating"].astype(str).apply(lambda x: x.lower() if x != "nan" else None)


In [656]:
df_reports.head(20)

,document_id,filename,date,provider,ticker,company_name,industry,paragraphs,target_price,rating
0,1,20161018_Needham_META_Facebook-_3Q16_Preview_R...,2016-10-11,Needham,META,Meta Platforms Inc.,Communication Services,[BUY INVESTMENT HIGHLIGHTS: $150.00 We raise o...,None,buy
1,2,20200807_Wells_Fargo_META_FB-_2.5B_Person_Plat...,2020-07-31,Wells Fargo,META,Meta Platforms Inc.,Communication Services,"[Internet Price Target Change Solid 2Q Beat, E...",300,v
2,3,20200918_Barclays_GM_General_Motors-_Time_to_t...,2020-09-14,Barclays,GM,General Motors Co.,Automobile,[Equity Research Instant Insights Stock Rating...,39.00,overweight
4,5,20200722_Barclays_BKNG_Booking_Holdings_Inc.-_...,2020-07-16,Barclays,BKNG,Booking Holdings Inc,Travel,[OVERWEIGHT Stock Rating Unchanged POSITIVE In...,None,overweight
5,6,20211104_Deutsche_Bank_X_US_Steel-_3Q21_EBITDA...,2021-10-28,Deutsche Bank,X,United States Steel,Materials,[Date North America Industrials 3Q21 FCF of $1...,50.00,buy
6,7,20190513_BTIG_PRGO_Perrigo_Company_plc.pdf,2019-05-08,BTIG,PRGO,Perrigo,Healthcare,"[After the close, Perrigo reported 1Q revenues...",None,none
7,8,20221118_JP_Morgan_HD_The_Home_Depot-_Beat_-_T...,2022-11-15,JP Morgan,HD,Home Depot Inc.,Consumer Discretionary,"[J P M O R G A N Overall, HD posted solid resu...",None,overweight
8,9,20211105_Wells_Fargo_XOM_XOM-_Post_Q3-21_Resul...,2021-10-29,Wells Fargo,XOM,Exxon Mobil Corp.,Energy,[We maintain our Overweight rating and raise o...,75.00,overweight
9,10,20210506_Wells_Fargo_META_FB-_Strong_1Q-_Navig...,2021-04-29,Wells Fargo,META,Meta Platforms Inc.,Communication Services,[Strong advertiser demand across Strong 1Q - M...,415.00,overweight
10,11,20191129_Barclays_MSFT_Microsoft-_Developers-_...,2019-11-22,Barclays,MSFT,Microsoft Corp.,Technology,[Equity Research Instant Insights Stock Rating...,168.00,overweight


# Get Date and Price for Dates

In [651]:
# Load and ensure 'Date' is in datetime format
df_performance_data = pd.read_csv("performance_data.csv")

# Convert 'Date' column to datetime and set it as index (with timezone awareness)
df_performance_data['Date'] = pd.to_datetime(df_performance_data['Date'], utc=True)
df_performance_data = df_performance_data.set_index('Date')

In [ ]:
print(df_reports[df_reports["date"]==])

Empty DataFrame
Columns: [document_id, filename, date, provider, ticker, company_name, industry, paragraphs, target_price, rating]
Index: []


In [652]:
df_saved_reports = df_reports

In [655]:
def get_stock_prices(ticker, start_date, end_date=None):
    try:
        # Ensure start_date and end_date are in the same timezone (UTC)
        start_date = pd.to_datetime(start_date, utc=True)
        if end_date is not None:
            end_date = pd.to_datetime(end_date, utc=True)
        else:
            end_date = start_date  # If no end_date, use start_date for a single day

        # Filter the data for the ticker and date range
        filtered_data = df_performance_data.loc[
            (df_performance_data.index >= start_date) & 
            (df_performance_data.index <= end_date), ticker]

        if not filtered_data.empty:
            # Return the maximum price within the filtered date range
            max_price = filtered_data.max()
            min_price = filtered_data.min()
            return round(float(max_price),2), round(float(min_price),2)
        else:
            # If no data available in the range, find the next available date using asof())
            next_available_data = df_performance_data[ticker].asof(start_date)
            if next_available_data is not None:
                return round(float(next_available_data),2), round(float(next_available_data),2)
            else:
                return round(float('nan')), round(float('nan'))  # Return NaN for missing data

    except KeyError:
        return float('nan'), float('nan')  # Return NaN for missing data
    except Exception as e:
        return float('nan'), float('nan')  # Return NaN for missing data

def calculate_prices(row):
    short_name = row['ticker']
    base_date = pd.to_datetime(row['date'], utc=True)
    print(f"base_date {base_date}")

    # Calculate prices
    row['start price'] = get_stock_prices(short_name, base_date)[0]
    row['one day after'] = get_stock_prices(short_name, base_date + pd.DateOffset(days=1))[0]
    row['max price after 3 months'], row["min price after 3 months"] = get_stock_prices(
        short_name, base_date, base_date + pd.DateOffset(months=3)
    )
    row['max price after 6 months'], row['min price after 6 months'] = get_stock_prices(
        short_name, base_date + pd.DateOffset(months=3), base_date + pd.DateOffset(months=6)
    )
    row['max price after 9 months'], row['min price after 9 months'] = get_stock_prices(
        short_name, base_date + pd.DateOffset(months=6), base_date + pd.DateOffset(months=9)
    )
    row['max price after 12 months'], row['min price after 12 months'] = get_stock_prices(
        short_name, base_date + pd.DateOffset(months=9), base_date + pd.DateOffset(months=12)
    )
    return row

# Apply the function to each row
df_saved_reports = df_saved_reports.apply(calculate_prices, axis=1)


base_date 2016-10-11 00:00:00+00:00
base_date 2020-07-31 00:00:00+00:00
base_date 2020-09-14 00:00:00+00:00
base_date 2020-07-16 00:00:00+00:00
base_date 2021-10-28 00:00:00+00:00
base_date 2019-05-08 00:00:00+00:00
base_date 2022-11-15 00:00:00+00:00
base_date 2021-10-29 00:00:00+00:00
base_date 2021-04-29 00:00:00+00:00
base_date 2019-11-22 00:00:00+00:00
base_date 2016-04-28 00:00:00+00:00
base_date 2015-02-26 00:00:00+00:00
base_date 2022-07-28 00:00:00+00:00
base_date 2022-11-01 00:00:00+00:00
base_date 2020-11-18 00:00:00+00:00
base_date 2014-09-16 00:00:00+00:00
base_date None


TypeError: unsupported operand type(s) for +: 'NoneType' and 'DateOffset'

In [530]:
df_saved_reports

,document_id,filename,date,provider,ticker,company_name,industry,paragraphs,target_price,rating,start price,one day after,max price after 3 months,min price after 3 months,max price after 6 months,min price after 6 months,max price after 9 months,min price after 9 months,max price after 12 months,min price after 12 months
0,1,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,2014-01-07,Needham,NVDA,NVIDIA Corp.,Technology,"[HOLD NA INVESTMENT HIGHLIGHTS: Last night, ah...",None,hold,0.38,0.39,0.45,0.36,0.47,0.43,0.48,0.42,0.51,0.40
1,2,20140116_Brean_Capital_URBN_URBN-_Ups_-_Downs-...,2014-01-09,Brean Capital,URBN,Urban Outfitters,Retail,"[Eric Beder , 212 702-6619 Danielle McCoy , 21...",None,hold,38.01,37.57,38.28,34.28,37.91,32.52,40.26,33.25,36.69,28.79
2,3,20140116_Brean_Capital_SIG_SIG-_Some_Pyrite_Mi...,2014-01-09,Brean Capital,SIG,Signet Jewelers,Retail,"[Eric Beder , 212 702-6619 Danielle McCoy , 21...",None,buy,61.33,59.50,89.38,58.41,93.87,81.56,99.53,85.25,110.82,87.36
3,4,20140122_Pivotal_Research_Group_GOOGL_GOOG-_Gr...,2014-01-21,Pivotal Research Group,GOOGL,Alphabet Inc.,Communication Services,[GOOGLE GOOG RATING: HOLD Target Price: $1070....,1070,hold,29.05,29.08,30.46,26.82,30.18,25.84,30.20,26.08,28.32,24.79


# Adjust target price for companies that had a split

In [531]:
df_saved_reports["target_price"] = (
    df_saved_reports["target_price"]
    .replace(",", "", regex=True)  # Remove commas
    .pipe(pd.to_numeric, errors="coerce")  # Convert to float; invalid values become NaN
)


In [539]:
import pandas as pd
from datetime import datetime

# Example stock splits dictionary (as provided)
stock_splits = {
    "AAPL": [
        {"split_ratio": 7, "ex_date": "2014-06-09"},
        {"split_ratio": 4, "ex_date": "2020-08-31"}
    ],
    "AMZN": [
        {"split_ratio": 20, "ex_date": "2022-06-06"}
    ],
    "TSLA": [
        {"split_ratio": 5, "ex_date": "2020-08-31"},
        {"split_ratio": 3, "ex_date": "2022-08-25"}
    ],
    "GOOGL": [
        {"split_ratio": 20, "ex_date": "2022-07-18"},
        {"split_ratio": 1.0027455, "ex_date": "2015-04-27"},
        {"split_ratio": 2.002, "ex_date": "2014-03-27"},
    ],
    "NVDA": [
        {"split_ratio": 10, "ex_date": "2024-06-10"},
        {"split_ratio": 4, "ex_date": "2021-07-20"}
    ],
    "NKE": [
        {"split_ratio": 2, "ex_date": "2015-12-24"}
    ],
    "V": [
        {"split_ratio": 4, "ex_date": "2015-03-19"}
    ],
    "MA": [
        {"split_ratio": 10, "ex_date": "2014-01-22"}
    ],
    "WMT": [
        {"split_ratio": 3, "ex_date": "2024-02-26"}
    ],
    "SBUX": [
        {"split_ratio": 2, "ex_date": "2015-04-09"}
    ],
    "UNP": [
        {"split_ratio": 2, "ex_date": "2014-06-09"}
    ],
    "UAA": [
        {"split_ratio": 2, "ex_date": "2014-04-15"}
    ],
    "NBR": [
        {"split_ratio": 0.02, "ex_date": "2020-04-23"}
    ],
}

# Define the function
def adjust_target_price(target_price, report_date, ticker, stock_splits):
    if pd.isna(target_price):
        return None  # Handle NaN target prices
    
    """ # Ensure report_date is a string
    if isinstance(report_date, pd.Timestamp):
        report_date = report_date.strftime("%Y-%m-%d")"""
    
    #report_date = datetime.strptime(report_date, "%Y-%m-%d")
    splits = stock_splits.get(ticker, [])
    
  

    for split in splits:
        split_date = datetime.strptime(split["ex_date"], "%Y-%m-%d").date()
        if split_date > report_date:
            target_price /= split["split_ratio"]
    
    return round(target_price,2)

# Apply the function to the DataFrame
df_saved_reports["adjusted_target_price"] = df_saved_reports.apply(
    lambda row: adjust_target_price(
        target_price=row["target_price"],
        report_date=row["date"],
        ticker=row["ticker"],
        stock_splits=stock_splits
    ),
    axis=1
)

df_saved_reports


,document_id,filename,date,provider,ticker,company_name,industry,paragraphs,target_price,rating,...,one day after,max price after 3 months,min price after 3 months,max price after 6 months,min price after 6 months,max price after 9 months,min price after 9 months,max price after 12 months,min price after 12 months,adjusted_target_price
0,1,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,2014-01-07,Needham,NVDA,NVIDIA Corp.,Technology,"[HOLD NA INVESTMENT HIGHLIGHTS: Last night, ah...",NaN,hold,...,0.39,0.45,0.36,0.47,0.43,0.48,0.42,0.51,0.40,NaN
1,2,20140116_Brean_Capital_URBN_URBN-_Ups_-_Downs-...,2014-01-09,Brean Capital,URBN,Urban Outfitters,Retail,"[Eric Beder , 212 702-6619 Danielle McCoy , 21...",NaN,hold,...,37.57,38.28,34.28,37.91,32.52,40.26,33.25,36.69,28.79,NaN
2,3,20140116_Brean_Capital_SIG_SIG-_Some_Pyrite_Mi...,2014-01-09,Brean Capital,SIG,Signet Jewelers,Retail,"[Eric Beder , 212 702-6619 Danielle McCoy , 21...",NaN,buy,...,59.50,89.38,58.41,93.87,81.56,99.53,85.25,110.82,87.36,NaN
3,4,20140122_Pivotal_Research_Group_GOOGL_GOOG-_Gr...,2014-01-21,Pivotal Research Group,GOOGL,Alphabet Inc.,Communication Services,[GOOGLE GOOG RATING: HOLD Target Price: $1070....,1070.0,hold,...,29.08,30.46,26.82,30.18,25.84,30.20,26.08,28.32,24.79,26.65


In [ ]:
#df_saved_reports.to_csv("reports.csv")

# Calculate performance of Target Prices

In [540]:
#If the target_price is below the current price, we need to check if it was below the target_price
def classifcy_performance(df):
    if df["target_price"] > df["start price"]:
        df["tp reached after 3 months"] = df["target_price"] <= df["max price after 3 months"]
        df["tp reached after 6 months"] = df["target_price"] <= df["max price after 6 months"]
        df["tp reached after 9 months"] = df["target_price"] <= df["max price after 9 months"]
        df["tp reached after 12 months"] = df["target_price"] <= df["max price after 12 months"]
    else:
        df["tp reached after 3 months"] = df["target_price"] >= df["min price after 3 months"]
        df["tp reached after 6 months"] = df["target_price"] >= df["min price after 6 months"]
        df["tp reached after 9 months"] = df["target_price"] >= df["min price after 9 months"]
        df["tp reached after 12 months"] = df["target_price"] <= df["min price after 12 months"]

    return df

df_saved_reports = df_saved_reports.apply(classifcy_performance, axis=1)

# Clean/Filter Paragraphs

In [ ]:
#df_saved_reports.to_csv("/Users/oskarroeske/Masterthesis/full_analysis/preprocessed_reports.csv")

In [541]:
# Explode the cleaned list into separate rows
df_exploded = df_saved_reports.explode('paragraphs').reset_index(drop=True)
df_exploded = df_exploded[["document_id","paragraphs"]]
df_exploded['paragraph_id'] = df_exploded.groupby('document_id').cumcount() + 1

In [543]:
df_reports

,document_id,filename,date,provider,ticker,company_name,industry,paragraphs,target_price,rating
0,1,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,2014-01-07,Needham,NVDA,NVIDIA Corp.,Technology,"[HOLD NA INVESTMENT HIGHLIGHTS: Last night, ah...",None,hold
1,2,20140116_Brean_Capital_URBN_URBN-_Ups_-_Downs-...,2014-01-09,Brean Capital,URBN,Urban Outfitters,Retail,"[Eric Beder , 212 702-6619 Danielle McCoy , 21...",None,hold
2,3,20140116_Brean_Capital_SIG_SIG-_Some_Pyrite_Mi...,2014-01-09,Brean Capital,SIG,Signet Jewelers,Retail,"[Eric Beder , 212 702-6619 Danielle McCoy , 21...",None,buy
3,4,20140122_Pivotal_Research_Group_GOOGL_GOOG-_Gr...,2014-01-21,Pivotal Research Group,GOOGL,Alphabet Inc.,Communication Services,[GOOGLE GOOG RATING: HOLD Target Price: $1070....,1070,hold


In [542]:
df_exploded

,document_id,paragraphs,paragraph_id
0,1,"HOLD NA INVESTMENT HIGHLIGHTS: Last night, ahe...",1
1,1,GameStream works hand in hand with the PC gami...,2
2,1,NVDA sought to bridge the gap between PC and m...,3
3,1,Recognizing the shift to advanced automobile f...,4
4,2,"Eric Beder , 212 702-6619 Danielle McCoy , 212...",1
5,2,Discussion Holiday comp of 3% was below the St...,2
6,2,Urban Outfitters Inc. URBNNASDAQ Brean Capital...,3
7,2,Urban Outfitters Inc. URBNNASDAQ Risks Urban O...,4
8,3,"Eric Beder , 212 702-6619 Danielle McCoy , 212...",1
9,3,Discussion Holiday November-December comp of 5...,2


In [ ]:
#df_exploded.to_csv("/Users/oskarroeske/Masterthesis/full_analysis/preprocessed_paragraphs.csv")

# Turn paragraphs into sentences

In [545]:
import spacy
import pandas as pd
from spacy.language import Language
from spacy.pipeline import Sentencizer

# Load spaCy language model
nlp = spacy.load("en_core_web_sm")

# Add a customized sentence boundary detection component to spaCy
@Language.component("custom_sentencizer")
def custom_sentencizer(doc):
    """
    Custom rules for sentence splitting, considering abbreviations, numbers, and uppercase logic.
    """
    for token in doc[:-1]:
        # Always split at '!' or '?'
        if token.text in ["!", "?"]:
            doc[token.i + 1].is_sent_start = True

        # Split at '.' only if not part of a number or abbreviation
        if token.text == ".":
            next_token = token.nbor(1) if token.i + 1 < len(doc) else None
            prev_token = token.nbor(-1) if token.i - 1 >= 0 else None
            
            # Check if next token starts a title-case word
            if next_token and next_token.is_title:
                doc[token.i + 1].is_sent_start = True
            
            # Prevent splitting for abbreviations
            if prev_token and prev_token.text.lower() in {"mr", "ms", "dr", "etc", "e.g", "adj", "sr"}:
                doc[token.i + 1].is_sent_start = False
            
            # Prevent splitting within numbers (e.g., 3.14)
            if prev_token and prev_token.like_num and next_token and next_token.like_num:
                doc[token.i + 1].is_sent_start = False

    return doc
# Add the custom sentencizer to the pipeline
nlp.add_pipe("custom_sentencizer", before="parser")

# Function to split paragraphs into sentences
def split_into_sentences(text):
    if not isinstance(text, str):
        return []  # Return an empty list for non-string values
    doc = nlp(text)
    return [sent.text.strip() for sent in doc.sents]

result = []

def check_sentence_validity(sentence):
    sentence = re.sub(r"\s+", " ", sentence).strip()
    has_verb = False
    has_subject = False

    sentence = nlp(sentence)
        
    for token in sentence:
        # Check for a verb
        if token.pos_ in {"VERB", "AUX"}:
            has_verb = True
        # Check for a subject
        if token.dep_ in {"nsubj", "nsubjpass"}:
            has_subject = True
 
    # If both a verb and a subject are found, the sentence is valid
    if has_verb and has_subject:
        return True
    return False

# Split each paragraph into sentences
for _, row in df_exploded.iterrows():
    doc_id = row['document_id']
    paragraph_id = row['paragraph_id']
    text = row['paragraphs']
    sentences = split_into_sentences(text)
    for sentence_id, sentence in enumerate(sentences, start=1):
        is_valid_sentence = check_sentence_validity(sentence)
        # Check if the sentence has more than 3 words
        if len(sentence.split()) >= 6 and is_valid_sentence:
            result.append({
                'document_id': doc_id,
                'paragraph_id': paragraph_id,
                'sentence_id': sentence_id,
                'sentence': sentence
            })

# Create a new DataFrame with the results
df_sentences = pd.DataFrame(result)

In [549]:
df_sentences["sentence"]

0     Last night, ahead of the 2014 CES, NVDA hosted...
1     The press conference, thematically centered ar...
2     We will be attending CES this week, and will p...
3     GameStream works hand in hand with the PC gami...
4     This demo was streamed through a server based ...
5     NVDAs second unveiling was its G-SYNC solution...
6     For G- SYNC, NVDA has partnered with leading g...
7     NVDA sought to bridge the gap between PC and m...
8     The Tegra K1, expected to be released 1H14, is...
9     The new processor also allows game developers ...
10    Recognizing the shift to advanced automobile f...
11    Additionally the VCM leverages the visual comp...
12    NVDA is currently Market Data in 4.5MM vehicle...
13    NVDA has spent heavily on its mobile applicati...
14    While NVDAs GPUs and level of integration are ...
15    We expect NVDA will seek to align itself with ...
16    Eric Beder , 212 702-6619 Danielle McCoy , 212...
17    The poor and negative comp performance in 

In [550]:
df_sentences

,document_id,paragraph_id,sentence_id,sentence
0,1,1,2,"Last night, ahead of the 2014 CES, NVDA hosted..."
1,1,1,3,"The press conference, thematically centered ar..."
2,1,1,4,"We will be attending CES this week, and will p..."
3,1,2,1,GameStream works hand in hand with the PC gami...
4,1,2,2,This demo was streamed through a server based ...
5,1,2,3,NVDAs second unveiling was its G-SYNC solution...
6,1,2,4,"For G- SYNC, NVDA has partnered with leading g..."
7,1,3,1,NVDA sought to bridge the gap between PC and m...
8,1,3,2,"The Tegra K1, expected to be released 1H14, is..."
9,1,3,3,The new processor also allows game developers ...


In [499]:
df_sentences.to_csv("/Users/oskarroeske/Masterthesis/full_analysis/preprocessed_sentences.csv")

In [ ]:
#df_reports.to_csv("paragraphs.csv")